Sample code for 1D transverse field Ising model (TFIM)

In [3]:
import numpy as np # library for numerics
import scipy.linalg as la # library for linear algebra
import random as rd # library for random numbers


# return parity of a 64bit integer m (by Andrew Shapira)
def parity(m):
        m ^= m>>1
        m ^= m>>2
        m = (m&0x1111111111111111) * 0x1111111111111111
        return (m>>60)&1


# initialize a complex vector[0:2**N-1]
def init_v(N):
        v = np.array([])
        for i in range(2**N):
                v = np.append(v,0.0+0.0j)
        return v

# initialize a complex random vector[0:2**N-1]
def rd_init_v(N):
        v = np.array([])
        norm = 0.0 + 0.0j
        for i in range(2**N):
                re = rd.random() - 0.5
                im = rd.random() - 0.5
                norm += re**2 + im**2
                v = np.append(v,re+im*1.0j)
        norm = np.sqrt(norm)
        return v/norm # normalized

Binary representaion for the spin at the $\ell$th site: $|1\rangle_{\ell} = |\uparrow\rangle_{\ell}$, $|0\rangle_{\ell} = |\downarrow\rangle_{\ell}$

$S=1/2$ spin operators acting on the $\ell$th site transform a wave function $c_1 |1\rangle_{\ell} + c_0|0\rangle_{\ell}$ as follows:

$\hat{S}^x_{\ell} \left(c_1 |1\rangle_{\ell} + c_0|0\rangle_{\ell} \right)=\frac{1}{2}c_0 |1\rangle_{\ell} + \frac{1}{2}c_1|0\rangle_{\ell}$, where $\hat{S}^x_{\ell}|1\rangle_{\ell} = +\frac{1}{2}|0\rangle_{\ell}$ and $\hat{S}^x_{\ell}|0\rangle_{\ell} = \frac{1}{2}|1\rangle_{\ell}$ 

$\hat{S}^y_{\ell} \left(c_1 |1\rangle_{\ell} + c_0|0\rangle_{\ell} \right)=-\frac{i}{2}c_0 |1\rangle_{\ell} + \frac{i}{2}c_1|0\rangle_{\ell}$, where $\hat{S}^y_{\ell}|1\rangle_{\ell} = +\frac{i}{2}|0\rangle_{\ell}$ and $\hat{S}^y_{\ell}|0\rangle_{\ell} = -\frac{i}{2}|1\rangle_{\ell}$ 



$\hat{S}^z_{\ell} \left(c_1 |1\rangle_{\ell} + c_0|0\rangle_{\ell} \right)=\frac{1}{2}c_0 |1\rangle_{\ell} - \frac{1}{2}c_1|0\rangle_{\ell}$, where $\hat{S}^z_{\ell}|1\rangle_{\ell} = +\frac{1}{2}|1\rangle_{\ell}$ and $\hat{S}^z_{\ell}|0\rangle_{\ell} = -\frac{1}{2}|0\rangle_{\ell}$ 

The ket vector representation is mapped to a 2 dimensional vector representation: $\hat{S}^y_{\ell}\left(c_1 |1\rangle_{\ell} + c_0|0\rangle_{\ell} \right)=-\frac{i}{2}c_0 |1\rangle_{\ell} + \frac{i}{2}c_1|0\rangle_{\ell} \rightarrow
S^{y}\left(\begin{array}{c}c_1\\c_0\\\end{array}\right)
=\frac{1}{2}\sigma^y \left(\begin{array}{c}c_1\\c_0\\\end{array}\right)$,
where the Pauli matrices: $\sigma^x=\left(\begin{array}{cc}0&1\\1&0\\\end{array}\right)$, 
$\sigma^y=\left(\begin{array}{cc}0&-i\\+i&0\\\end{array}\right)$, 
$\sigma^z=\left(\begin{array}{cc}+1&0\\0&-1\\\end{array}\right)$

In [4]:
# define spin operators for S=1/2
class spin_operators:
        # function returns j, c1 that satsfies S^x_ell c0|i> = c1|j>
        def SxI(self,ell,i,c0):
                icomb = 2**ell
                j = i^icomb
                return j, (0.5 + 0.0j)*c0
        # function returns j, c1 that satsfies S^y_ell c0|i> = c1|j>
        def SyI(self,ell,i,c0):
                icomb = 2**ell
                ibit  = i&icomb
                ibit  = parity(ibit) # obtain ellth bit (parity is not necessary)
                j = i^icomb
                return j, ((-1.0)**ibit) * (0.0 - 0.5j)*c0
        # function returns j, c1 that satsfies S^z_ell c0|i> = c1|j>
        def SzI(self,ell,i,c0):
                icomb = 2**ell
                ibit  = i&icomb
                ibit  = parity(ibit) # obtain ellth bit (parity is not necessary)
                return i, ((-1.0)**ibit) * (-0.5 + 0.0j)*c0

The following class defines a 1D TFIM, $\hat{H}=J\sum_{i=0}^{L-1}
  \hat{S}^{z}_{{\rm mod}(i+1,L)}\hat{S}^{z}_{i}
  -\Gamma \sum_{i=0}^{L-1} \hat{S}^{x}_{i}$:

In [12]:
# a class inherits from the class spin operators
class hamiltonian(spin_operators):
        # constructor for a L site 1D Heisenberg model with exchange coupling J
        def __init__(self,L,J,Gamma):
                self.L = L
                self.J = J
                self.Gamma = Gamma
        def param(self):
                print("L=",self.L,", J=",self.J,", Gamma=",self.Gamma)
        def multiply(self,v0,v1):
                for i in range(2**(self.L)):
                        c0 = v0[i]
                        # define 1D TFIM
                        for j in range(self.L):
                                j1, c1 = self.SxI(j,i,c0)
                                v1[j1] -= self.Gamma*c1
                                j1, c1 = self.SzI(j,i,c0)
                                j2, c2 = self.SzI((j+1)%self.L,j1,c1)
                                v1[j2] += self.J*c2

Set Hamiltonian parameters

In [18]:
L = 6
J = 1.0
Gamma = 0.1
ham = hamiltonian(L,J,Gamma)
ham.param()

L= 6 , J= 1.0 , Gamma= 0.1


Construct Hamiltonian matrix $H$ and fully diagonalize $H$. The lowest eigenvalue is printed. When $L\geq 12$, it tooks bit long.

In [19]:
H = np.zeros((2**L,2**L),dtype=np.complex)
for i in range(2**L):
        v0 = init_v(L)
        v1 = init_v(L)
        v0[i] = 1.0 + 0.0j
        ham.multiply(v0,v1)
        for j in range(2**L):
                H[j][i] = v1[j]
eig_vals,eig_vec = la.eig(H)
eig_id = np.argsort(eig_vals)
eig_vals = eig_vals[eig_id]
eig_vec = eig_vec[:,eig_id]
print(eig_vals[0:5].real)

[-1.51504176 -1.51503401 -0.68222134 -0.65677644 -0.65677644]


Please complete the Lanczos steps below.

Tips:
1. The inner product of v0 and v1 is given by np.dot(v0.conjugate(),v1).
2. The eigenvalues and eigenvectors of H_Ks[i,j] (0$\leq$i,j$\leq$m) is given through eig_vals,eig_vec = la.eig(H_Ks[0:m+1,0:m+1]).

In [20]:
N_lmax = 32 # upper limit of the number of the Lanczos step
v0 = init_v(L)
w  = init_v(L)
v1 = rd_init_v(L) # initial vector
beta = 0.0 + 0.0j
H_Ks = np.zeros((N_lmax+1,N_lmax+1),dtype=np.complex)
for m in range(N_lmax):
        w = -beta *  v0
        ham.multiply(v1,w) # w += H * v1
        alpha = np.dot(w.conjugate(),v1)
        w += -alpha * v1
        beta = np.sqrt(abs(np.dot(w.conjugate(),w)))
        v0 = v1
        v1 = w / beta
        H_Ks[m,m] = alpha
        H_Ks[m,m+1] = beta
        H_Ks[m+1,m] = beta
        eig_vals,eig_vec = la.eig(H_Ks[0:m+1,0:m+1])
        eig_id = np.argsort(eig_vals)
        eig_vals = eig_vals[eig_id]
        eig_vec = eig_vec[:,eig_id]
        if m < 2:
          print(m,eig_vals[0].real)
        else:
          print(m,eig_vals[0].real,eig_vals[1].real,eig_vals[2].real)

0 -0.008392869460009209
1 -0.5526467556965107
2 -0.8751677746711348 0.053309033713210524 1.100791418962493
3 -1.48592075729758 -0.5091146688904519 0.49411994053102054
4 -1.511635529750449 -0.5340533708997582 0.09667547283522351
5 -1.5149575205366046 -0.5954079535560869 -0.43797914768532525
6 -1.5150295594205636 -0.6127514829269385 -0.46200340065387807
7 -1.5150382338997337 -0.6460407196020037 -0.5245693000336509
8 -1.515038396055067 -0.6548059910369414 -0.5401549137521645
9 -1.5150384172004285 -0.6721421766749184 -0.5810957854220147
10 -1.515038417504322 -0.6750664685649338 -0.587850571827728
11 -1.515038417564865 -0.7330181775408062 -0.6626165606864137
12 -1.515038417612127 -1.2029009987535924 -0.6746604217654499
13 -1.51503841850135 -1.4989668931171438 -0.6776956558133904
14 -1.515038424400412 -1.5128710239119023 -0.6782736043546596
15 -1.515038835330732 -1.515003156895811 -0.679935872529629
16 -1.5150406342890281 -1.5150317731759693 -0.6805097680329492
17 -1.5150417363828041 -1.5150